In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

## Read the Data

In [2]:
data = pd.read_csv("train.csv")
data.head()

,Tarih,Dağıtılan Enerji (MWh)
0,2018-01-01 00:00:00,1593.944216
1,2018-01-01 01:00:00,1513.933887
2,2018-01-01 02:00:00,1402.612637
3,2018-01-01 03:00:00,1278.527266
4,2018-01-01 04:00:00,1220.697701


In [3]:
future_data = pd.read_csv("future.csv").drop('Unnamed: 0' , axis = 1)
future_data.head()

,Tarih,Dağıtılan Enerji (MWh)
0,2022-08-01 00:00:00,NaN
1,2022-08-01 01:00:00,NaN
2,2022-08-01 02:00:00,NaN
3,2022-08-01 03:00:00,NaN
4,2022-08-01 04:00:00,NaN


# Describe the Data

In [4]:
data.shape

(40152, 2)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40152 entries, 0 to 40151
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Tarih                   40152 non-null  object 
 1   Dağıtılan Enerji (MWh)  40152 non-null  float64
dtypes: float64(1), object(1)
memory usage: 627.5+ KB


In [6]:
data['Tarih'] = pd.to_datetime(data['Tarih'])
future_data['Tarih'] = pd.to_datetime(future_data['Tarih'])
data.set_index('Tarih', inplace=True)
future_data.set_index('Tarih' , inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 40152 entries, 2018-01-01 00:00:00 to 2022-07-31 23:00:00
Data columns (total 1 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Dağıtılan Enerji (MWh)  40152 non-null  float64
dtypes: float64(1)
memory usage: 627.4 KB


In [7]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Dağıtılan Enerji (MWh),40152.0,1836.805287,426.066085,870.18328,1499.165048,1813.409221,2129.407998,3633.105297


# Plot the Data

# Modelling

## Train | Test Split

In [8]:
import tensorflow as tf

# Seed ayarlamak
seed_value = 34
tf.keras.utils.set_random_seed(seed_value)

In [9]:
X_test_temp = data.loc[data.index > '2022-07-23 23:00:00'].copy()
X_train_temp = data.loc[data.index <= '2022-07-23 23:00:00'].copy()

X_val_temp = X_train_temp[-96:]
X_train_temp = X_train_temp[:-96]

In [10]:
print(f"X_train max : {X_train_temp.max()}")
print(f"X_val max : {X_val_temp.max()}")
print(f"X_test max : {X_test_temp.max()}")

print(f"X_train min : {X_train_temp.min()}")
print(f"X_val min : {X_val_temp.min()}")
print(f"X_test min : {X_test_temp.min()}")

X_train max : Dağıtılan Enerji (MWh)    3633.105297
dtype: float64
X_val max : Dağıtılan Enerji (MWh)    2812.085678
dtype: float64
X_test max : Dağıtılan Enerji (MWh)    3079.546897
dtype: float64
X_train min : Dağıtılan Enerji (MWh)    870.18328
dtype: float64
X_val min : Dağıtılan Enerji (MWh)    1596.356554
dtype: float64
X_test min : Dağıtılan Enerji (MWh)    1594.214702
dtype: float64


## Scaling

In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
data_scaled = scaler.fit_transform(data)
train_scaled = scaler.transform(X_train_temp)
test_scaled = scaler.transform(X_test_temp)

val_scaled = scaler.transform(X_val_temp)

In [12]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler(feature_range=(0,1))
# train_scaled = scaler.fit_transform(X_train_temp)
# test_scaled = scaler.transform(X_test_temp)

# val_scaled = scaler.transform(X_val_temp)

## Reshape

Buraya dikkat !

In [13]:
def packager(train_scaled, val_scaled, test_scaled, window_size=24):
    X_train, y_train, X_val, y_val, X_test, y_test = [], [], [], [], [], []

    for i in range(window_size, len(train_scaled)):
        X_train.append(train_scaled[i-window_size:i, 0])
        y_train.append(train_scaled[i, 0])

    for i in range(window_size, len(val_scaled)):
        X_val.append(val_scaled[i-window_size:i, 0])
        y_val.append(val_scaled[i, 0])

    for i in range(window_size, len(test_scaled)):
        X_test.append(test_scaled[i-window_size:i, 0])
        y_test.append(test_scaled[i, 0])

    X_train, y_train = np.array(X_train), np.array(y_train)
    X_val, y_val = np.array(X_val), np.array(y_val)
    X_test, y_test = np.array(X_test), np.array(y_test)

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    X = np.concatenate((X_train, X_val, X_test), axis=0)
    y = np.concatenate((y_train, y_val, y_test), axis=0)

    return X_train, y_train, X_val, y_val, X_test, y_test, X, y

In [14]:
X_train, y_train, X_val, y_val, X_test, y_test, X, y = packager(train_scaled, val_scaled, test_scaled)

In [15]:
print(f"X_train Shape : {X_train.shape}")
print(f"X_test Shape : {X_test.shape}")
print(f"X_val Shape : {X_val.shape}")
print(f"X Shape : {X.shape}")

X_train Shape : (39840, 24, 1)
X_test Shape : (168, 24, 1)
X_val Shape : (72, 24, 1)
X Shape : (40080, 24, 1)


# Eval Metric

In [16]:
def eval_metrics(y_true , y_pred):
    from sklearn.metrics import r2_score , mean_absolute_error , mean_squared_error , mean_absolute_percentage_error
    
    # MAPE hesaplama
    mape = mean_absolute_percentage_error(y_true, y_pred)
    
    # r2 hesaplama
    r2 = r2_score(y_true , y_pred)
    
    # mae hesaplama
    mae = mean_absolute_error(y_true , y_pred)

    # rmse hesaplama
    mse = mean_squared_error(y_true,y_pred)**0.5
    
    print(f"""
          Mape Score : {mape}
          R2 Score : {r2}
          MAE Score : {mae}
          MSE Score : {mse}
          """)
    
def eval_plot(y_true , y_pred):
    tests = pd.DataFrame(data = y_true , columns=['Real Values'] , index = X_test[:-24].index)
    preds = pd.DataFrame(data = y_pred , columns=['Predicts'] , index = future_data[:-24].index)
    compare = pd.concat([tests[:-24], preds] , axis= 1)
    print(compare.plot())
    
def eval_df (y_true , y_pred):
    compare = pd.DataFrame({'Real Values': y_true, 'Predicts': y_pred}, index=future_data[:-24].index)
    print(compare)
    
def create_submission(future_preds, num):
    submission_df = pd.DataFrame({'Tarih': future_data.index, 'Dağıtılan Enerji (MWh)': future_preds})
    filename = 'submission{}.csv'.format(num)
    submission_df.to_csv(filename, index=False)
    globals()['submission{}'.format(num)] = submission_df
    
def preds_plot(data , future_data):
    # Gerçek değerleri mavi renkte çizdir
    plt.plot(data, color='blue', label='Gerçek Değerler')

    # Tahmin edilen değerleri yeşil renkte çizdir
    plt.plot(future_data, color='green', label='Tahminler')

    # Eksenleri ve grafik başlığını belirle
    plt.title('Gerçek Değerler ve Tahminler')
    plt.xlabel('Saat')
    plt.ylabel('Değer')
    plt.legend()

    # Grafikleri göster
    plt.show()

# Modelling Time

## Manual

In [ ]:
# from keras.models import Sequential
# from keras.layers import LSTM, Dense

# # Model oluşturma
# model = Sequential()
# model.add(LSTM(50, input_shape=(24, 1), return_sequences= True))
# model.add(LSTM(25))
# model.add(Dense(1))
# model.compile(loss='mse', optimizer='adam')

# # Modeli eğitme
# model.fit(X_train , y_train, epochs= 10, validation_data=(X_val , y_val))

# # Modeli değerlendirme
# mse = model.evaluate(X_test , y_test)
# print('Test MSE: %.3f' % mse)

## Automatic

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error , mean_absolute_percentage_error
import numpy as np
import pandas as pd

In [18]:
# Define the function for creating and training the LSTM model
def create_lstm_model(hidden_size, initial_num_unit, learning_rate = 0.001, epoch=20, batch_size = 1 , window_size = 24 , unit_style= 'decrease' , save=False):
    if unit_style == "decrease":
        return_sequences = [True] * (hidden_size - 1) + [False]
        model = Sequential()
        for i in range(hidden_size):
            if i == 0:
                model.add(LSTM(initial_num_unit, input_shape=(window_size, 1), return_sequences=return_sequences[i]))
            else:
                model.add(LSTM(initial_num_unit // 2**i, return_sequences=return_sequences[i]))
        model.add(Dense(1))
        optimizer = Adam(learning_rate=learning_rate)
        model.compile(loss='mean_squared_error', optimizer=optimizer)
        if save:
            return_sequences = [True] * (hidden_size - 1) + [False]
            model = Sequential()
            for i in range(hidden_size):
                if i == 0:
                    model.add(LSTM(initial_num_unit, input_shape=(window_size, 1), return_sequences=return_sequences[i]))
                else:
                    model.add(LSTM(initial_num_unit // 2**i, return_sequences=return_sequences[i]))
            model.add(Dense(1))
            optimizer = Adam(learning_rate=learning_rate)
            model.compile(loss='mean_squared_error', optimizer=optimizer)
            
            
            # sc = MinMaxScaler(feature_range=(0,1))
            # data_scaled = sc.fit_transform(data)
            # train_scaled = sc.transform(X_train_temp)
            # test_scaled = sc.transform(X_test_temp)

            # val_scaled = sc.transform(X_val_temp)
            
            # X_train, y_train, X_val, y_val, X_test, y_test, X, y = packager(train_scaled, val_scaled, test_scaled)
            
            
            model.fit(X, y, epochs=epoch, batch_size=batch_size, verbose=1)
            y_pred = model.predict(X_test)
            org_y_pred = scaler.inverse_transform(y_pred.reshape(-1,1))
            org_y_test = scaler.inverse_transform(y_test.reshape(-1,1))
            
            testScore = mean_absolute_percentage_error(org_y_test, org_y_pred)
            
            model.save(f'../models/lstm_model_{hidden_size}_{initial_num_unit}_{learning_rate}_{epoch}_{round(testScore, 4)}.h5')
        else:
            model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epoch, batch_size=batch_size, verbose=1)
            return model
        
    elif unit_style == 'increase':
        return_sequences = [True] * (hidden_size - 1) + [False]
        model = Sequential()
        for i in range(hidden_size):
            if i == 0:
                model.add(LSTM(initial_num_unit, input_shape=(window_size, 1), return_sequences=return_sequences[i]))
            else:
                model.add(LSTM(initial_num_unit * 2**i, return_sequences=return_sequences[i]))
        model.add(Dense(1))
        optimizer = Adam(learning_rate=learning_rate)
        model.compile(loss='mean_squared_error', optimizer=optimizer)
        if save:
            model.fit(X, y, epochs=epoch, batch_size=batch_size, verbose=1)
            model.save(f'../models/lstm_model_{hidden_size}_{initial_num_unit}_{learning_rate}_{epoch}.h5')
        else:
            model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epoch, batch_size=batch_size, verbose=1)
            return model
        
    elif unit_style == "same":
        return_sequences = [True] * (hidden_size - 1) + [False]
        model = Sequential()
        for i in range(hidden_size):
            if i == 0:
                model.add(LSTM(initial_num_unit, input_shape=(window_size, 1), return_sequences=return_sequences[i]))
            else:
                model.add(LSTM(initial_num_unit, return_sequences=return_sequences[i]))
        model.add(Dense(1))
        optimizer = Adam(learning_rate=learning_rate)
        model.compile(loss='mean_squared_error', optimizer=optimizer)
        if save:
            model.fit(X, y, epochs=epoch, batch_size=batch_size, verbose=1)
            model.save(f'../models/lstm_model_{hidden_size}_{initial_num_unit}_{learning_rate}_{epoch}.h5')
        else:
            model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epoch, batch_size=batch_size, verbose=1)
            return model

In [19]:
# Define empty lists to store model results
hidden_sizes = []
initial_units_list = []
learning_rates_list = []
epochs_list = []
train_scores = []
val_scores = []
test_scores = []

# Define hyperparameters to tune
hidden_layer_sizes = [2,3]
initial_num_units = [32,16]
learning_rates = [0.001, 0.01]
epochs = [10, 20, 30]

# Define the number of epochs and batch size to use in training
batch_size = 1

# Define the number of previous time steps to use as input features
window_size = 24
        
        
for hidden_size in hidden_layer_sizes:
    for initial_num_unit in initial_num_units:
        for learning_rate in learning_rates:
            for epoch in epochs:
                print(f"Hidden size : {hidden_size} , Initial Num neurons : {initial_num_unit} , Learning Rate : {learning_rate} , Epoch : {epoch}")
                model = create_lstm_model(hidden_size, initial_num_unit, learning_rate , batch_size= batch_size , epoch = epoch , unit_style= 'decrease' )
                
                
                trainPredict = model.predict(X_train)
                valPredict = model.predict(X_val)
                testPredict = model.predict(X_test)
                org_y_train_pred = scaler.inverse_transform(trainPredict)
                org_y_val_pred = scaler.inverse_transform(valPredict)
                org_y_test_pred = scaler.inverse_transform(testPredict)
                
                org_y_train = scaler.inverse_transform(y_train.reshape(-1,1))
                org_y_val = scaler.inverse_transform(y_val.reshape(-1,1))
                org_y_test = scaler.inverse_transform(y_test.reshape(-1,1))
                
                trainScore = mean_absolute_percentage_error(org_y_train, org_y_train_pred)
                valScore = mean_absolute_percentage_error(org_y_val, org_y_val_pred)
                testScore = mean_absolute_percentage_error(org_y_test_pred, org_y_test_pred)


                # Append results to lists
                hidden_sizes.append(hidden_size)
                initial_units_list.append(initial_num_unit)
                learning_rates_list.append(learning_rate)
                epochs_list.append(epoch)
                train_scores.append(trainScore)
                val_scores.append(valScore)
                test_scores.append(testScore)
                print(f"Hidden size : {hidden_size} , Initial Num neurons : {initial_num_unit} , Learning Rate : {learning_rate} , Epoch : {epoch} , Unit Style : {'Decrease'} ")
                print(f"Train Score : {trainScore} , Val Score : {valScore} , Test Score : {testScore}")
                print("--------------------------------------------------------------------------------")
                if testScore < 0.02 :
                    create_lstm_model(hidden_size , initial_num_unit , learning_rate , batch_size = batch_size , epoch = epoch, save= True)

# Create dataframe from results
results_df = pd.DataFrame({
    'HiddenSize': hidden_sizes,
    'Initial Num Neurons': initial_units_list,
    'LearningRate': learning_rates_list,
    'Epoch': epochs_list,
    'TrainScore': train_scores,
    'valScore': val_scores,
    'TestScore': test_scores
}).sort_values(by='TestScore' , ascending=False)

# Print dataframe
results_df.style.format("{:.2%}").background_gradient(cmap="Blues")
print(results_df)

Hidden size : 2 , Initial Num neurons : 32 , Learning Rate : 0.001 , Epoch : 10
Epoch 1/10
39840/39840 [==============================] - 139s 3ms/step - loss: 9.0405e-04 - val_loss: 4.9125e-04
Epoch 2/10
39840/39840 [==============================] - 138s 3ms/step - loss: 3.8235e-04 - val_loss: 2.1770e-04
Epoch 3/10
39840/39840 [==============================] - 138s 3ms/step - loss: 3.1059e-04 - val_loss: 2.2983e-04
Epoch 4/10
39840/39840 [==============================] - 133s 3ms/step - loss: 2.6888e-04 - val_loss: 2.0374e-04
Epoch 5/10
39840/39840 [==============================] - 128s 3ms/step - loss: 2.4937e-04 - val_loss: 1.1818e-04
Epoch 6/10
39840/39840 [==============================] - 131s 3ms/step - loss: 2.3667e-04 - val_loss: 2.0523e-04
Epoch 7/10
39840/39840 [==============================] - 129s 3ms/step - loss: 2.2240e-04 - val_loss: 1.9802e-04
Epoch 8/10
39840/39840 [==============================] - 131s 3ms/step - loss: 2.1479e-04 - val_loss: 1.7078e-04
Epoch 9/

KeyboardInterrupt: 

In [ ]:
# Print dataframe
results_df.style.format("{:.2%}").background_gradient(cmap="Blues")
results_df

# Test | Evaluation

In [ ]:
# Done